In [2]:
CUDA_VISIBLE_DEVICES=0

In [2]:
import torch
import torch.nn as nn
import os
import logging
import numpy as np
import random
from tqdm import tqdm
import time
import pandas as pd

from transformers import LongformerModel, AutoTokenizer, LongformerForSequenceClassification, LongformerForMultipleChoice
from transformers import AutoTokenizer, RobertaModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report

logging.basicConfig(filename=f'./logs/train_{time.asctime().replace(" ","_")}.log', filemode='w', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Create a logger object
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a stream handler to print log messages to the console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

torch.manual_seed(40)
np.random.seed(40)
random.seed(40)
torch.cuda.manual_seed(40)
torch.backends.cudnn.deterministic = True

/data1/debajyoti/colie/colenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define the path to the CSV file
train_csv_file = "/data1/debajyoti/colie/train.csv"
val_csv_file = "/data1/debajyoti/colie/valid.csv"
test_csv_file = "/data1/debajyoti/colie/test.csv"

# Read the CSV file
train_labels = pd.read_csv(train_csv_file)
val_labels = pd.read_csv(val_csv_file)
test_labels = pd.read_csv(test_csv_file)
test_labels

,BOOK_id
0,7616_1.txt
1,7616_2.txt
2,7616_3.txt
3,7616_4.txt
4,7616_5.txt
...,...
143025,5677_92.txt
143026,5677_93.txt
143027,5677_94.txt
143028,5677_95.txt


In [4]:
train_labels.BOOK_id[0]

'27993_1.txt'

In [5]:
# Define the path to the train folder
train_folder = "/data1/debajyoti/colie/train/train/"
# Define the path to the validation folder
val_folder = "/data1/debajyoti/colie/valid/valid/"
# Define the path to the test folder
test_folder = "/data1/debajyoti/colie/test/test/"



def create_df(folder, label):
    # Initialize empty lists to store the data
    text_data = []
    labels = []
    for index in label.index:
        # filename = df_labels.BOOK_id[index]
        # print(filename)
        # print(df_labels['BOOK_id'][index], df_labels['Epoch'][index])
        file_name = label['BOOK_id'][index]  # Assuming 'File Name' is the column name for the file names in the CSV

        # Construct the file path
        file_path = os.path.join(folder, file_name)

        # Read the text from the file
        with open(file_path, 'r', encoding='ISO-8859-1') as file:
            text = file.read()

        # Append the text and label to the respective lists
        text_data.append(text)
        labels.append(label['Epoch'][index].strip())  # Assuming 'Label' is the column name for the labels in the CSV
        # break
    return text_data, labels

def create_df_test(folder, label):
    # Initialize empty lists to store the data
    text_data = []
    # labels = []
    for index in label.index:
        # filename = df_labels.BOOK_id[index]
        # print(filename)
        # print(df_labels['BOOK_id'][index], df_labels['Epoch'][index])
        file_name = label['BOOK_id'][index]  # Assuming 'File Name' is the column name for the file names in the CSV

        # Construct the file path
        file_path = os.path.join(folder, file_name)

        # Read the text from the file
        with open(file_path, 'r', encoding='ISO-8859-1') as file:
            text = file.read()

        # Append the text and label to the respective lists
        text_data.append(text)
        # labels.append(label['Epoch'][index].strip())  # Assuming 'Label' is the column name for the labels in the CSV
        # break
    return text_data

train_data, train_label = create_df(train_folder, train_labels)
val_data, val_label = create_df(val_folder, val_labels)
test_data = create_df_test(test_folder, test_labels)

# Create a dataframe from the lists
train = pd.DataFrame({'text': train_data, 'label': train_label})
val = pd.DataFrame({'text': val_data, 'label': val_label})
test = pd.DataFrame({'text': test_data})
print(train.head(), val.head(), test.head())
print(train.shape, val.shape, test.shape)

                                                text      label
0  rifle; Ivan's was a double-barrelled shot-gun ...  Viktorian
1  upon the track of the bear. After following it...  Viktorian
2  to pull him out with their hands--even had the...  Viktorian
3  a slight sparkle of scientific conceit, "this ...  Viktorian
4  bears with a white ring round their necks? Yes...  Viktorian                                                 text      label
0  kind good morning, and returned her hearty emb...  Viktorian
1  sky, and of the moon, which clothed the old pi...  Viktorian
2  left Rome for Augsburg, my mind being much exc...  Viktorian
3  thoughts some of the old melodies he knew by h...  Viktorian
4  "But," said Henry, "is it not possible that th...  Viktorian                                                 text
0  "Alas, poor girl!" said I, "I fear that her ha...
1  to divide her attention between the said garco...
2  visitor's disposition to gallantry. However, s...
3  says Juvenal, "'M

In [7]:
label_dic = {'Romanticism':0,
            'Viktorian':1,
            'Modernism':2,
            'PostModernism':3,
            'OurDays':4}
train['label'] = train['label'].map(label_dic)
val['label'] = val['label'].map(label_dic)
# test['label'] = test['label'].map(label_dic)

In [8]:
# Length of text
def length (txt):
    length = len(txt.split())
    return length

txt_length = train['text'].apply(lambda x: length(x))
print(txt_length.sort_values(ascending = False))

483268    1128
483267    1068
521384    1065
483265    1034
81542     1020
          ... 
470405       1
130188       1
217335       1
351867       1
368135       1
Name: text, Length: 546210, dtype: int64


In [9]:
val['label'].value_counts()

label
1    16938
2    14848
3     1713
4     1600
0     1158
Name: count, dtype: int64

In [10]:
# model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
# tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [11]:
max_length= 500
class CustomDataset(Dataset):
    def __init__(self, tokenizer, df):
        # Initialize thetokenizer
        self.tokenizer = tokenizer

        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Get the text and label from the dataframe
        text = self.df.iloc[index]['text']
        label = self.df.iloc[index]['label']

        # Tokenize the text and convert it to input IDs
        inputs = self.tokenizer(
            text,
            None,
            add_special_tokens=False,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )


        # Return the input IDs and label as PyTorch tensors
        return {
            'input_ids': inputs['input_ids'][0],
            'attention_mask': inputs['attention_mask'][0],
            # 'token_type_ids': inputs['token_type_ids'][0],
            'label': torch.tensor(label, dtype=torch.int64),
        }

# datasetclass = CustomDataset(tokenizer, train)
train_dataset = CustomDataset(tokenizer, train)
val_dataset = CustomDataset(tokenizer, val)
# test_dataset = CustomDataset(tokenizer, test)

# DataLoader
batch_size = 256
train_dataloader = tqdm(DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=64))
val_dataloader = tqdm(DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=64))
# test_dataloader = tqdm(DataLoader(test_dataset, batch_size=batch_size, shuffle=False))

  0%|          | 0/2134 [00:00<?, ?it/s]

In [12]:
class TransformerModel(nn.Module):
    def __init__(self, num_labels):
        super(TransformerModel, self).__init__()
        
        self.roberta = RobertaModel.from_pretrained("roberta-base")
        # self.Longformer = LongformerModel.from_pretrained("allenai/longformer-base-4096")
        # Freeze all layers except the top 1
        for param in self.roberta.parameters():
            param.requires_grad = False

        # Unfreeze the parameters of the top 1 layers
        for param in self.roberta.encoder.layer[-1:].parameters():
            param.requires_grad = True
        
        # self.xlnet.resize_token_embeddings(num_tokens)
        # self.transformer_encoder = TransformerEncoder(TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads), num_layers=num_layers)
        #self.transformer_decoder = TransformerDecoder(TransformerDecoderLayer(d_model=hidden_size, nhead=num_heads), num_layers=num_layers)
        #self.transformer = Transformer(nhead=16, num_encoder_layers=6, num_decoder_layers = 6)
        self.decoder = nn.Linear(self.roberta.config.hidden_size, num_labels) 
        # self.fc1 = nn.Linear(num_tokens, 2)
        # self.fc2 = nn.Linear(num_tokens, 2)
        # self.fc3 = nn.Linear(num_tokens, 5)
        # self.num_classes = num_classes
        # self.classifiers = nn.ModuleList([nn.Linear(self.roberta.config.hidden_size, num_classes[i]) for i in range(len(num_classes))])
        # self.classifiers = nn.ModuleList([nn.Linear(num_tokens, num_classes[i]) for i in range(len(num_classes))])
        # self.tanh = nn.Tanh()

    def forward(self, input_ids, attention_mask):  # src = [bsz, seq_len]
        long_output = self.roberta(input_ids=input_ids).pooler_output
        # print(long_output.shape)
        # roberta_outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        # last_hidden_state = outputs.last_hidden_state # Shape: (batch_size, sequence_len, hidden_size)
        # src_embedded = last_hidden_state
        # src_embedded = self.roberta.embeddings(src) # Use RoBERTa model to embed source sequence output: [bsz, seq_len, features,i.e. hidden_dim] [20, 100, 768]
        # print("shape of roberta embeddings:", src_embedded.shape)
        #tgt_embedded = self.roberta.embeddings(tgt) # Use RoBERTa model to embed target sequence
        # src_embedded = src_embedded # output: [bsz, seq_len, features] 
        # src_embedded = torch.cat([t1,t2,t3, src_embedded],1)

        # t1 = torch.cat(src_embedded.size(0) * [t1])
        # t2 = torch.cat(src_embedded.size(0) * [t2])
        # t3 = torch.cat(src_embedded.size(0) * [t3])
        # t = torch.stack([t1,t2,t3], dim=1)
        # task_embedded = torch.cat([t, src_embedded],1)  # output shape: [bsz, seq_len, features] [8, 203, 768]

        # memory = self.transformer_encoder(src_embedded)  # output shape: [bsz, seq_len, features] [8, 203, 768]
        # print("shape after transformer encoder layer:", memory.shape)
        #output = self.transformer_decoder(tgt_embedded, memory)
        #print("shape after transformer decoder layer:", output.shape)

        output = self.decoder(long_output)  # output shape: [bsz, seq_len, vocab_size] [8, 203, 50k]
        # print("shape after transformer decoder layer:", output.shape, output.dtype)
        # task1_output = self.fc1(output[:,0,:])
        # task2_output = self.fc2(output[:,1,:])
        # task3_output = self.fc3(output[:,2,:num_classes])
        # ae_output = output[:,len(self.num_classes):,:]
        # ae_output = output[:,:,:]
        # print("shape after final linear layer:", output.shape)
        # task_logits = [classifier(pooled_output) for classifier in self.classifiers]
        # task_logits = []

        # pooled_outputs = [output[:,i,:] for i in range(len(self.num_classes))] # output shape : [bsz, 1, vocab_size]

        # for classifier, pooled_output in zip(self.classifiers, pooled_outputs):
        #     # pooled_output = self.tanh(pooled_output)
        #     logits = classifier(pooled_output)
        #     task_logits.append(logits)
        
        return output

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [13]:
num_labels = 5

model = TransformerModel(num_labels).to(device)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# num_epochs = 5
learning_rate = 2e-4
class_weights = torch.tensor([12.0, 1.0, 1.0, 8.0, 11.0]).to(device)

# Set optimizer and learning rate scheduler
criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [15]:
def get_labels(logit, targets):
    """
    Calculate accuracy and macro F1 score for each class
    """
    # pos = list(task_dict.keys()).index(task_name)
    # mask = torch.arange(targets.shape[0]).to(device)
    # task_idx = mask[targets[:,pos] != 99]
    output = logit
    true_label = targets
    # print("shapes for label:", output.shape, true_label.shape)
    pred_label = torch.argmax(output, dim=1).flatten().tolist()
    true_label = true_label.flatten().tolist()


    return pred_label, true_label

In [15]:
current_train_loss = []

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 1
    start_time = time.time()
    num_batches = len(train_dataset) // batch_size
    for batch, i in enumerate(train_dataloader):
        data, mask, targets = i.values()
        data = data.to(device)
        mask = mask.to(device)
        targets = targets.to(device)
        # print(data.dtype)        
        # print(data.shape)
        # task_logits, ae_output = model(data)
        output = model(data, mask)
        # t1_out, t2_out, t3_out, auto_output = model(data, t1, t2, t3)
        # loss = custom_loss(logits_task1, logits_task2, logits_task3, targets)
        # print("shape:", data.shape, targets.flatten().shape)
        # print("outputtype:", output.dtype, targets.flatten().dtype)
        # print(output)
        # targets = targets.float()
        loss = criterion(output, targets.flatten())


        optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            # ppl = np.exp(cur_loss)
            # print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
            #         f'lr {lr:02.7f} | ms/batch {ms_per_batch:5.2f} | '
            #         f'loss {cur_loss:5.5f}')
            total_loss = 0
            start_time = time.time()
        
        # if batch == 100:
        #     break
    current_train_loss.append(cur_loss)


In [16]:
def evaluate(model: nn.Module) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    # src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        predictions = []
        true_labels = []
        for batch, i in enumerate(val_dataloader):
            data, mask, targets = i.values()
            data = data.to(device)
            mask = mask.to(device)
            targets = targets.to(device)
            seq_len = data.size(1)
            # logits_task1, logits_task2, logits_task3, ae_output = model(data, mask)
            # task_logits, ae_output = model(data)
            # task_logits, ae_output = model(data, mask)
            output = model(data, mask)
            # t1_out, t2_out, t3_out, auto_output = model(data, t1, t2, t3)
            # loss = custom_loss(logits_task1, logits_task2, logits_task3, targets)
            # loss = custom_loss(logits_task1, logits_task2, logits_task3, ae_output, data, targets)
            loss = criterion(output, targets.flatten())

            total_loss += seq_len * loss.item()

            #get the labels for classification report
            pred_label, true_label = get_labels(output, targets)
            predictions.extend(pred_label)
            true_labels.extend(true_label)
            # if batch == 100:
            #     break

    # Compute overall classification report
    logging.info(f"\n Scores:")
    acc = accuracy_score(true_labels, predictions)
    logging.info(f"\n Accuracy:{acc}")
    logging.info(f"\n {classification_report(true_labels, predictions)}")
    return total_loss / (len(val_dataset) - 1), acc


In [17]:
logging.info(f"#"* 89)
logging.info(f"\n DESCRIPTION-> \n logic: roberta(finetune last layer) + linear_layer + loss_reweighting (epochs=50), model: {tokenizer.name_or_path}, lr:{learning_rate}, max_seq_length: {max_length}")
logging.info('#' * 89)

2023-07-13 19:57:09,815 - INFO - #########################################################################################
2023-07-13 19:57:09,818 - INFO - 
 DESCRIPTION-> 
 logic: roberta(finetune last layer) + linear_layer + loss_reweighting (epochs=50), model: roberta-base, lr:0.0002, max_seq_length: 500
2023-07-13 19:57:09,820 - INFO - #########################################################################################


In [18]:
best_val_acc = 0
current_val_loss = []   # for plotting graph of val_loss
epochs = 50
early_stop_thresh = 3

tempdir = '/data1/debajyoti/colie/.temp/'
best_model_params_path = os.path.join(tempdir, f"best_model_params_{time.asctime().replace(' ','_')}.pt")

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss, accuracy = evaluate(model)
    current_val_loss.append(val_loss)
    # val_ppl = np.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    logging.info('-' * 89)
    logging.info(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
        f'valid loss {val_loss:5.5f}')
    logging.info('-' * 89)

    if accuracy > best_val_acc:
        best_val_acc = accuracy
        best_epoch = epoch
        torch.save(model.state_dict(), best_model_params_path)
    elif epoch - best_epoch > early_stop_thresh:
        logging.info("Early stopped training at epoch %d" % epoch)
        break  # terminate the training loop

    scheduler.step()
model.load_state_dict(torch.load(best_model_params_path)) # load best model states


100%|██████████| 142/142 [2:02:45<00:00, 51.87s/it]
2023-07-13 21:59:50,945 - INFO - 
 Scores:
2023-07-13 21:59:50,958 - INFO - 
 Accuracy:0.604462586535014
2023-07-13 21:59:51,002 - INFO - 
               precision    recall  f1-score   support

           0       0.18      0.40      0.24      1158
           1       0.76      0.62      0.68     16938
           2       0.74      0.66      0.70     14848
           3       0.22      0.38      0.28      1713
           4       0.12      0.25      0.16      1600

    accuracy                           0.60     36257
   macro avg       0.40      0.46      0.41     36257
weighted avg       0.68      0.60      0.63     36257

2023-07-13 21:59:51,003 - INFO - -----------------------------------------------------------------------------------------
2023-07-13 21:59:51,004 - INFO - | end of epoch   1 | time: 7361.17s | valid loss 2.11808
2023-07-13 21:59:51,005 - INFO - -------------------------------------------------------------------------

<All keys matched successfully>

In [17]:
test

,text
0,"""Alas, poor girl!"" said I, ""I fear that her ha..."
1,to divide her attention between the said garco...
2,"visitor's disposition to gallantry. However, s..."
3,"says Juvenal, ""'Mors sola fatetur Quantula sin..."
4,him out in that back passage; the outer door i...
...,...
143025,"be hard for anyone to do anything dignified, w..."
143026,"Wilson, and could not bring himself to think t..."
143027,"give them a chance, the same as everybody else..."
143028,political convention in which he declared that...


In [18]:
class CustomDataset_test(Dataset):
    def __init__(self, tokenizer, df):
        # Initialize the tokenizer
        self.tokenizer = tokenizer

        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Get the text and label from the dataframe
        text = self.df.iloc[index]['text']
        # label = self.df.iloc[index]['label']

        # Tokenize the text and convert it to input IDs
        inputs = self.tokenizer(
            text,
            None,
            add_special_tokens=False,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )


        # Return the input IDs and label as PyTorch tensors
        return {
            'input_ids': inputs['input_ids'][0],
            'attention_mask': inputs['attention_mask'][0],
            # 'token_type_ids': inputs['token_type_ids'][0],
            # 'label': torch.tensor(label, dtype=torch.int64),
        }

test_dataset = CustomDataset_test(tokenizer, test)

# DataLoader
test_dataloader = tqdm(DataLoader(test_dataset, batch_size=batch_size, shuffle=False))

In [21]:
predictions = []

# Evaluate model on test data
model.eval()
with torch.no_grad():
    for batch_idx, batch in enumerate(test_dataloader):
        data, mask = batch.values()
        data = data.to(device)
        mask = mask.to(device)
        # targets = targets.to(device)
        seq_len = data.size(1)
        # logits_task1, logits_task2, logits_task3, ae_output = model(data, mask)
        # task_logits, ae_output = model(data)
        # task_logits, ae_output = model(data, mask)
        output = model(data, mask)
        #get the labels for classification report
        pred_label = torch.argmax(output, dim=1).flatten().tolist()
        predictions.extend(pred_label)
        
        # if batch == 400:
        #     break

# # Compute overall classification report
# logging.info(f"\n Scores:")
# logging.info(f"\n {classification_report(true_labels, predictions)}")
# return total_loss / (len(val_dataset) - 1)

100%|██████████| 559/559 [33:50<00:00,  3.63s/it]


In [22]:
test_labels["Epoch"] = predictions
# test_labels.to_csv('file_name.csv', index=False)

In [23]:
label_dic = {0:'Romanticism',
            1:'Viktorian',
            2:'Modernism',
            3:'PostModernism',
            4:'OurDays'}
test_labels['Epoch'] = test_labels['Epoch'].map(label_dic)
test_labels

,BOOK_id,Epoch
0,7616_1.txt,Viktorian
1,7616_2.txt,Viktorian
2,7616_3.txt,Viktorian
3,7616_4.txt,Viktorian
4,7616_5.txt,Viktorian
...,...,...
143025,5677_92.txt,Viktorian
143026,5677_93.txt,Modernism
143027,5677_94.txt,Modernism
143028,5677_95.txt,Modernism


In [24]:
test_labels.to_csv('/data1/debajyoti/colie/submission/submission_4.csv', index=False)

In [1]:
# Import libraries

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from scipy.sparse import hstack

In [ ]:
# Create the feature extractor using trigrams
trigram_vectorizer = CountVectorizer(ngram_range=(3, 3))

# Transform the data into trigram feature vectors
X_trigrams = trigram_vectorizer.fit_transform(X)

In [ ]:
# Create the feature extractor using bag of words
bow_vectorizer = CountVectorizer()

# Transform the data into bag of words feature vectors
X_bow = bow_vectorizer.fit_transform(X)

In [ ]:
# Concatenate the trigram features and bag of words features
X_features = hstack([X_trigrams, X_bow])

In [ ]:
# Perform feature selection using chi2 #2KFH
k = 400 # Select the top k features 
feature_selector = SelectKBest(chi2, k=k)
X_selected = feature_selector.fit_transform(X_features, y)

In [ ]:
# Create the Naive Bayes classifier
nb_classifier = MultinomialNB()

In [ ]:
# Perform cross-validation and print the classification report
cv_results = cross_val_score(nb_classifier, X_selected, y, cv=10, scoring='accuracy')
classification_report_cv = classification_report(y, cross_val_predict(nb_classifier, X_selected, y, cv=10))
mean_accuracy = cv_results.mean()

In [24]:
# Import libraries

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from scipy.sparse import hstack


In [19]:
train_df = pd.concat([train, val], ignore_index=True, axis=0)
train

,text,label
0,rifle; Ivan's was a double-barrelled shot-gun ...,1
1,upon the track of the bear. After following it...,1
2,to pull him out with their hands--even had the...,1
3,"a slight sparkle of scientific conceit, ""this ...",1
4,bears with a white ring round their necks? Yes...,1
...,...,...
546205,"the manner described in the text, might lay cl...",2
546206,"surnamed, answered, Ã¢ÂÂNa, na, there are na...",2
546207,that of Themis. My informant was Alexander Kei...,2
546208,"a heavy blow. cloyed a dud, stolen a rag. coll...",2


In [21]:
# Assuming you have separate dataframes for train, val, and test
train_data = train_df['text']
train_labels = train_df['label']

# val_data = val['text']
# val_labels = val['label']

test_data = test['text']

In [22]:
vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(train_data)
# val_vectors = vectorizer.transform(val_data)
test_vectors = vectorizer.transform(test_data)

In [23]:
# Create the feature extractor using trigrams
trigram_vectorizer = CountVectorizer(ngram_range=(3, 3))

# Transform the data into trigram feature vectors
train_vectors_trigrams = trigram_vectorizer.fit_transform(train_data)

In [25]:
# Concatenate the trigram features and bag of words features
X_features = hstack([train_vectors, train_vectors_trigrams])

In [41]:
# Perform feature selection using chi2 #2KFH
k = 2400 # Select the top k features 
feature_selector = SelectKBest(chi2, k=k)
X_selected = feature_selector.fit_transform(X_features, train_labels)

In [42]:
classifier = MultinomialNB()
classifier.fit(X_selected, train_labels)

MultinomialNB()

In [43]:
# Perform cross-validation and print the classification report
cv_results = cross_val_score(classifier, X_selected, train_labels, cv=10, scoring='accuracy')
classification_report_cv = classification_report(train_labels, cross_val_predict(classifier, X_selected, train_labels, cv=10))
mean_accuracy = cv_results.mean()
mean_accuracy

0.5512879305459965

In [44]:
print(classification_report_cv)

              precision    recall  f1-score   support

           0       0.17      0.67      0.27     21319
           1       0.66      0.53      0.59    257303
           2       0.70      0.61      0.65    252936
           3       0.21      0.41      0.27     23852
           4       0.23      0.17      0.20     27057

    accuracy                           0.55    582467
   macro avg       0.39      0.48      0.40    582467
weighted avg       0.62      0.55      0.57    582467



In [14]:
test_predictions = classifier.predict(test_vectors)

In [15]:
test_predictions

array([1, 1, 0, ..., 2, 2, 2])

In [16]:
test_labels["Epoch"] = test_predictions

In [17]:
label_dic = {0:'Romanticism',
            1:'Viktorian',
            2:'Modernism',
            3:'PostModernism',
            4:'OurDays'}
test_labels['Epoch'] = test_labels['Epoch'].map(label_dic)
test_labels

,BOOK_id,Epoch
0,7616_1.txt,Viktorian
1,7616_2.txt,Viktorian
2,7616_3.txt,Romanticism
3,7616_4.txt,Romanticism
4,7616_5.txt,Romanticism
...,...,...
143025,5677_92.txt,Modernism
143026,5677_93.txt,Modernism
143027,5677_94.txt,Modernism
143028,5677_95.txt,Modernism


In [18]:
test_labels.to_csv('/data1/debajyoti/colie/submission/submission_6.csv', index=False)